In [1]:
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
import numpy as np
import string
import re
from functools import reduce

stopwords_set = set(stopwords.words('english'))
stopwords_set = {'i', 'shan', 'just', 'how', 'each', 'out', 'themselves', 'their', 'before', 'were', 'very', 'as', 'further', 'his', 'a', 'once', 'youve', 'y', 'is', 'shouldve', 'youll', 'on', 'd', 'm', 'under', 'haven', 'which', 'only', 'them', 'was', 'by', 'needn', 'whom', 'that', 'when', 's', 'isn', 'its', 'wasn', 'in', 'we', 'theirs', 'those', 'this', 'having', 'and', 'ain', 'most', 'up', 'off', 'being', 'aren', 'shouldn', 'ourselves', 'from', 'down', 'herself', 'her', 'you', 'are', 'its', 'who', 'the', 'here', 'where', 'your', 'youd', 'she', 'didn', 'weren', 'about', 'has', 'our', 'an', 'yourselves', 'or', 'hasn', 'again', 'while', 'does', 'him', 'shes', 'above', 'below', 'itself', 'to', 'through', 'will', 'couldn', 'hers', 'they', 'doing', 'because', 'he', 'what', 'such', 'youre', 'too', 'should', 'ours', 'then', 'himself', 'all', 'of', 'mightn', 'between', 'now', 'against', 'some', 'with', 'until', 'am', 'other', 'at', 'can', 'over', 'mustn', 'wouldn', 'do', 'for', 'after', 'hadn', 'me', 'been', 'same', 'doesn', 'my', 'these', 'll', 'did', 'had', 'it', 'so', 'ma', 'during', 'than', 'o', 'yourself', 'own', 'have', 're', 've', 'be', 'why', 't', 'there', 'more', 'won', 'yours', 'few', 'into', 'thatll', 'any', 'myself', 'both', 'don', 'if'}

In [2]:
fname = 'project_2_train/' + 'data 2_train.csv'

In [3]:
sentence_weights = []
X, Y, X1 = [], [], []

In [4]:
f = open(fname, 'r')

In [5]:
for i, line in enumerate(f):
    if i != 0:
        ''' Splitting the columns based on comma - since it is csv'''
        columns = line.split(',')

        ''' The comma in the actual sentence was represented as [commma] because of csv format, replace that '''
        columns[1] = columns[1].replace('[comma]', ',')
        columns[2] = columns[2].replace('[comma]', ',')

        ''' Some aspect terms are mic in the sentence where computer/mic is present '''
        columns[1] = columns[1].replace('-', ' ')
        columns[2] = columns[2].replace('-', ' ')
        columns[1] = columns[1].replace('/', ' ')
        columns[2] = columns[2].replace('/', ' ')

        ''' NOt used anymore'''
        tokenizer = RegexpTokenizer(r'\w+') # doesn't work if the special char is in the token
        # columns[2] = tokenizer.tokenize(columns[2].lower())

        '''Tokenize the words'''
        columns[1] = word_tokenize(columns[1])
        columns[2] = word_tokenize(columns[2])

        for j, elem in enumerate(columns[1]):
            '''Remove special characters'''
            columns[1][j] = re.sub('[^0-9a-zA-Z]+', '', elem).lower()
            ''' didn't is tokenized as did n't hence following code '''
            if columns[1][j] == 'nt':
                columns[1][j] = 'not'

        for j, elem in enumerate(columns[2]):
            '''Remove special characters'''
            columns[2][j] = re.sub('[^0-9a-zA-Z]+', '', elem).lower()
            ''' didn't is tokenized as did n't hence following code '''
            if columns[2][j] == 'nt':
                columns[2][j] = 'not'

        '''Remove empty string tokens'''
        columns[1] = [x.strip() for x in columns[1] if x.strip() != '']
        columns[2] = [x.strip() for x in columns[2] if x.strip() != '']

        ''' Remove stop words '''
        columns[1] = [word for word in columns[1] if word not in stopwords_set]
        columns[1] = [word for word in columns[1] if word not in stopwords_set]
        columns[2] = [word for word in columns[2] if word not in stopwords_set]

        columns[1] = ' '.join(columns[1])
        columns[2] = ' '.join(columns[2])

        ''' The aspect term location given is not proper - hence extracting the location by ourselves '''
        columns[3] = []
        ''' Finds the location of the aspect term in the sentence '''
        for m in re.finditer(columns[2], columns[1]):
            columns[3].append([m.start(), m.end()])
            ''' since some aspect terms are not surrounded by spaces, hence below code
            NOT USED ANYMORE
            '''
            if m.start() != 0 and columns[1][m.start() - 1] != ' ':
                # columns[1] = columns[1][:m.start()] + ' ' + columns[1][m.start():]
                pass

        ''' Because of adding space in above code, the position is messed up, hence redoing it. '''
        columns[3] = []

        for m in re.finditer(columns[2], columns[1]):
            columns[3].append([m.start(), m.end()])

        ''' Tokenizing the words again '''
        columns[1] = columns[1].split(' ')
        columns[2] = columns[2].split(' ')
        ''' New column to specify the aspect term location in the list '''
        columns.append([])
        # columns[5] = []
        ''' for multiple positions of aspect term '''
        for elem in columns[3]:
            if elem[0] == 0: #start index of the 1st occurance of the aspect term
                columns[5].append([0])
            else:
                temp_len = elem[0] # assign the start position of the aspect term
                for j, tokens in enumerate(columns[1]):
                    temp_len -= (len(tokens) + 1) # Counting the words till the position - +1 for blank space
                    if temp_len == 0: # Reached the aspect word
                        columns[5].append([j+1])
                        break
            for k in range(len(columns[2])-1): # if multiple words in the aspect term - tag the following words in the sentence
                columns[5][-1].append(columns[5][-1][-1]+1)

        ''' if not found the aspect term location in the list then do '''
        if len(columns[5]) < 1:
            continue

            # pass

        word_wt_list = [[elem, 0] for elem in columns[1]]
        ''' Assigning weights to every word based on the distance from the aspect term '''
        for j, elem in enumerate(columns[5]):
            for k, word in enumerate(columns[1]):
                if k < elem[0]: # Word left to the aspect term
                    dist = abs(elem[0] - k)
                    if word_wt_list[k][1] < 1/dist:
                        word_wt_list[k][1] = 1/dist
                elif k > elem[-1]: # word right to the aspect term : Aspect term can have multiple words
                    dist = abs(k - elem[-1])
                    if word_wt_list[k][1] < 1/dist:
                        word_wt_list[k][1] = 1/dist
            for aspect_word_loc in elem:
                word_wt_list[aspect_word_loc][1] = 1.5

        ''' For duplicate words - if it's aspect term - add the weights, if not, then take the weight which is greater, make the other weight as zero '''
        for j, word_1 in enumerate(word_wt_list):
            for k, word_2 in enumerate(word_wt_list[j+1:]):
                if word_1[0] == word_2[0]:
                    if word_1[1] == 1.5 and word_2[1] == 1.5:
                        # print(line)
                        # print(word_wt_list)
                        word_1[1] += 1.5
                        word_2[1] = 0
                    elif word_1[1] < word_2[1]:
                        # print(line)
                        # print(word_wt_list)
                        word_1[1] = word_2[1]
                        word_2[1] = 0
                    elif word_1[1] > word_2[1]:
                        # print(line)
                        # print(word_wt_list)
                        word_2[1] = 0
                    elif word_1[1] == word_2[1]:
                        word_2[1] = 0


        ''' For removing duplicate words - they have weight zero now'''
        word_wt_list = [elem for elem in word_wt_list if elem[1] != 0]

        if len(columns[2]) > 1 and len(columns[5]) > 1:
            # print(line)
            # print(columns)
            # print(word_wt_list)
            pass

        if columns[0] == '2911_0':
            # print(line)
            # print(word_wt_list)
            pass
        if i == 100:
            # break
            pass

        '''Removing new line character from columns[4] -  the polarity column'''
#         columns[4] = columns[4].rstrip('\n')
        X.append(columns[1])
        X1.append(' '.join(columns[1]))

        Y.append(columns[4])
        sentence_weights.append(word_wt_list)

'success'

'success'

In [6]:
'''Building vocabulary and counting word occurances'''
X_reduced = reduce(lambda x1, x2: x1 + x2, X)
vocab = list(set(X_reduced))
print(vocab[0])
weight_v = np.zeros_like(vocab, dtype = np.float_)
weight_x = []


potato


In [7]:
pf = open('weightxy_data2.csv', 'w')

pf.write(','.join(vocab))
pf.write(',class_')
pf.write('\n')

1

In [8]:
for i, sentence in enumerate(sentence_weights):
    for word in sentence:
        v_index = vocab.index(word[0])
        weight_v[v_index] = word[1]

    weight_x.append(weight_v)
    if len(sentence) != weight_v[np.where(weight_v > 0)].shape[0]:
        print(sentence)
        print(len(sentence))
        print(weight_v[np.where(weight_v > 0)])

    ''' code to write the data to file '''
    temp = [str(j) for j in weight_v.tolist()]
    pf.write(','.join(temp))
    pf.write(','+str(Y[i]))
    pf.write('\n')
    weight_v = np.zeros_like(weight_v)

In [9]:
pf.close()
weight_x = np.array(weight_x)

print(weight_x.shape)

f.close()


(3602, 3613)


In [10]:
''' Training data converted. NOt testing data '''
test_fname = 'project_2_test/' + 'Data-2_test.csv'
test_sentence_weights = []
test_X, test_X1 = [], []
test_data_Y = []

In [11]:
test_f = open(test_fname, 'r')

In [12]:
for i, line in enumerate(test_f):
    if i != 0:
        ''' Splitting the columns based on comma - since it is csv'''
        columns = line.split(',')

        ''' The comma in the actual sentence was represented as [commma] because of csv format, replace that '''
        columns[1] = columns[1].replace('[comma]', ',')
        columns[2] = columns[2].replace('[comma]', ',')

        ''' Some aspect terms are mic in the sentence where computer/mic is present '''
        columns[1] = columns[1].replace('-', ' ')
        columns[2] = columns[2].replace('-', ' ')
        columns[1] = columns[1].replace('/', ' ')
        columns[2] = columns[2].replace('/', ' ')

        ''' NOt used anymore'''
        tokenizer = RegexpTokenizer(r'\w+') # doesn't work if the special char is in the token
        # columns[2] = tokenizer.tokenize(columns[2].lower())

        '''Tokenize the words'''
        columns[1] = word_tokenize(columns[1])
        columns[2] = word_tokenize(columns[2])

        for j, elem in enumerate(columns[1]):
            '''Remove special characters'''
            columns[1][j] = re.sub('[^0-9a-zA-Z]+', '', elem).lower()
            ''' didn't is tokenized as did n't hence following code '''
            if columns[1][j] == 'nt':
                columns[1][j] = 'not'

        for j, elem in enumerate(columns[2]):
            '''Remove special characters'''
            columns[2][j] = re.sub('[^0-9a-zA-Z]+', '', elem).lower()
            ''' didn't is tokenized as did n't hence following code '''
            if columns[2][j] == 'nt':
                columns[2][j] = 'not'

        '''Remove empty string tokens'''
        columns[1] = [x.strip() for x in columns[1] if x.strip() != '']
        columns[2] = [x.strip() for x in columns[2] if x.strip() != '']

        ''' Remove stop words '''
        columns[1] = [word for word in columns[1] if word not in stopwords_set]
        columns[1] = [word for word in columns[1] if word not in stopwords_set]
        columns[2] = [word for word in columns[2] if word not in stopwords_set]

        columns[1] = ' '.join(columns[1])
        columns[2] = ' '.join(columns[2])

        ''' The aspect term location given is not proper - hence extracting the location by ourselves '''
        columns[3] = []
        ''' Finds the location of the aspect term in the sentence '''
        for m in re.finditer(columns[2], columns[1]):
            columns[3].append([m.start(), m.end()])
            ''' since some aspect terms are not surrounded by spaces, hence below code
            NOT USED ANYMORE
            '''
            if m.start() != 0 and columns[1][m.start() - 1] != ' ':
                # columns[1] = columns[1][:m.start()] + ' ' + columns[1][m.start():]
                pass

        ''' Because of adding space in above code, the position is messed up, hence redoing it. '''
        columns[3] = []

        for m in re.finditer(columns[2], columns[1]):
            columns[3].append([m.start(), m.end()])

        ''' Tokenizing the words again '''
        columns[1] = columns[1].split(' ')
        columns[2] = columns[2].split(' ')
        ''' New column to specify the aspect term location in the list '''
        columns.append([])
        columns.append([]) 
        # columns[5] = []
        ''' for multiple positions of aspect term '''
        for elem in columns[3]:
            if elem[0] == 0: #start index of the 1st occurance of the aspect term
                columns[5].append([0])
            else:
                temp_len = elem[0] # assign the start position of the aspect term
                for j, tokens in enumerate(columns[1]):
                    temp_len -= (len(tokens) + 1) # Counting the words till the position - +1 for blank space
                    if temp_len == 0: # Reached the aspect word
                        columns[5].append([j+1])
                        break
            for k in range(len(columns[2])-1): # if multiple words in the aspect term - tag the following words in the sentence
                try:
                    columns[5][-1].append(columns[5][-1][-1]+1)
                except:
                    print(line)
                    print(columns)
                    pass
    
        
        word_wt_list = [[elem, 0] for elem in columns[1]]
        ''' run only if aspect term found, otherwise the weights are zero '''
        if len(columns[5]) >= 1:
            
            ''' Assigning weights to every word based on the distance from the aspect term '''
            for j, elem in enumerate(columns[5]):
                for k, word in enumerate(columns[1]):
                    if k < elem[0]: # Word left to the aspect term
                        dist = abs(elem[0] - k)
                        if word_wt_list[k][1] < 1/dist:
                            word_wt_list[k][1] = 1/dist
                    elif k > elem[-1]: # word right to the aspect term : Aspect term can have multiple words
                        dist = abs(k - elem[-1])
                        if word_wt_list[k][1] < 1/dist:
                            word_wt_list[k][1] = 1/dist
                for aspect_word_loc in elem:
                    word_wt_list[aspect_word_loc][1] = 1.5

        ''' For duplicate words - if it's aspect term - add the weights, if not, then take the weight which is greater, make the other weight as zero '''
        for j, word_1 in enumerate(word_wt_list):
            for k, word_2 in enumerate(word_wt_list[j+1:]):
                if word_1[0] == word_2[0]:
                    if word_1[1] == 1.5 and word_2[1] == 1.5:
                        # print(line)
                        # print(word_wt_list)
                        word_1[1] += 1.5
                        word_2[1] = 0
                    elif word_1[1] < word_2[1]:
                        # print(line)
                        # print(word_wt_list)
                        word_1[1] = word_2[1]
                        word_2[1] = 0
                    elif word_1[1] > word_2[1]:
                        # print(line)
                        # print(word_wt_list)
                        word_2[1] = 0
                    elif word_1[1] == word_2[1]:
                        word_2[1] = 0

        if len(columns[5]) >= 1:
            ''' For removing duplicate words - they have weight zero now'''
            word_wt_list = [elem for elem in word_wt_list if elem[1] != 0]
        else: 
            print(line)
            print(columns)
            print(word_wt_list)

        if len(columns[2]) > 1 and len(columns[5]) > 1:
            # print(line)
            # print(columns)
            # print(word_wt_list)
            pass

        if columns[0] == '2911_0':
            # print(line)
            # print(word_wt_list)
            pass
        if i == 100:
            # break
            pass

        test_X.append(columns[1])
#         test_data_Y.append(columns[4]) # TODO comment this for final
        test_X1.append(' '.join(columns[1]))

        test_sentence_weights.append(word_wt_list)

In [13]:
test_weight_v = np.zeros_like(vocab, dtype = np.float_)
test_weight_x = []

In [14]:
pf = open('final_test2.csv', 'w')

pf.write(','.join(vocab))

# pf.write(',class_') # comment this for final TODO


pf.write('\n')
temp_i = -1

In [15]:
for i, sentence in enumerate(test_sentence_weights):
    for word in sentence:
        try:
            v_index = vocab.index(word[0])
            test_weight_v[v_index] = word[1]
        except:
            temp_i = i
            print(word[0], i)
    test_weight_x.append(test_weight_v)
    if temp_i == i:
        print(sentence)
    if len(sentence) != test_weight_v[np.where(test_weight_v > 0)].shape[0]:
        print(sentence)
        print(len(sentence))
        print(test_weight_v[np.where(test_weight_v > 0)])

    ''' code to write the data to file '''
    temp = [str(j) for j in test_weight_v.tolist()]
    pf.write(','.join(temp))
#     pf.write(','+str(test_data_Y[i])) # comment this for final TODO
    pf.write('\n')
    test_weight_v = np.zeros_like(test_weight_v)

reccomend 0
orange 0
[['reccomend', 0.14285714285714285], ['fried', 1.5], ['pork', 0.2], ['dumplings', 0.25], ['orange', 0.3333333333333333], ['chicken', 0.5], ['beef', 1.0], ['rice', 1.5]]
[['reccomend', 0.14285714285714285], ['fried', 1.5], ['pork', 0.2], ['dumplings', 0.25], ['orange', 0.3333333333333333], ['chicken', 0.5], ['beef', 1.0], ['rice', 1.5]]
8
[0.25 1.   1.5  0.2  1.5  0.5 ]
sharp 1
[['staff', 1.5], ['sharp', 1.0], ['look', 0.5], ['good', 0.3333333333333333]]
[['staff', 1.5], ['sharp', 1.0], ['look', 0.5], ['good', 0.3333333333333333]]
4
[0.5        0.33333333 1.5       ]
hawaiian 2
sweets 2
[['best', 0.5], ['dessert', 1.0], ['chocolate', 1.5], ['peanut', 1.5], ['butter', 1.5], ['tart', 1.5], ['not', 1.0], ['particularly', 0.5], ['hawaiian', 0.3333333333333333], ['but', 0.25], ['small', 0.2], ['world', 0.16666666666666666], ['comes', 0.14285714285714285], ['sweets', 0.125]]
[['best', 0.5], ['dessert', 1.0], ['chocolate', 1.5], ['peanut', 1.5], ['butter', 1.5], ['tart', 1

peg 107
clientele 107
rowdy 107
mouthed 107
commuters 107
dc 107
concert 107
knicks 107
game 107
[['service', 0.05555555555555555], ['however', 0.058823529411764705], ['peg', 0.0625], ['two', 0.06666666666666667], ['quality', 0.07142857142857142], ['food', 0.07692307692307693], ['horrible', 0.08333333333333333], ['bartenders', 0.09090909090909091], ['clientele', 0.1], ['part', 0.1111111111111111], ['rowdy', 0.125], ['loud', 0.14285714285714285], ['mouthed', 0.16666666666666666], ['commuters', 0.2], ['could', 0.25], ['explain', 0.3333333333333333], ['bad', 0.5], ['attitudes', 1.0], ['staff', 1.5], ['getting', 1.0], ['loaded', 0.5], ['ac', 0.3333333333333333], ['dc', 0.25], ['concert', 0.2], ['knicks', 0.16666666666666666], ['game', 0.14285714285714285]]
[['service', 0.05555555555555555], ['however', 0.058823529411764705], ['peg', 0.0625], ['two', 0.06666666666666667], ['quality', 0.07142857142857142], ['food', 0.07692307692307693], ['horrible', 0.08333333333333333], ['bartenders', 0.090

creamy 199
taramasalata 199
greek 199
yogurt 199
cuccumber 199
dill 199
pitas 199
[['creamy', 0.2], ['appetizers', 0.25], ['taramasalata', 0.3333333333333333], ['eggplant', 0.5], ['salad', 1.0], ['greek', 1.5], ['yogurt', 1.5], ['cuccumber', 1.5], ['dill', 1.5], ['garlic', 1.5], ['taste', 1.0], ['excellent', 0.5], ['warm', 0.3333333333333333], ['pitas', 0.25]]
[['creamy', 0.2], ['appetizers', 0.25], ['taramasalata', 0.3333333333333333], ['eggplant', 0.5], ['salad', 1.0], ['greek', 1.5], ['yogurt', 1.5], ['cuccumber', 1.5], ['dill', 1.5], ['garlic', 1.5], ['taste', 1.0], ['excellent', 0.5], ['warm', 0.3333333333333333], ['pitas', 0.25]]
14
[0.5        0.25       1.         1.         1.5        0.33333333
 0.5       ]
churrasco 200
[['best', 0.25], ['things', 0.3333333333333333], ['order', 0.5], ['grill', 1.0], ['churrasco', 1.5], ['ribs', 1.0]]
[['best', 0.25], ['things', 0.3333333333333333], ['order', 0.5], ['grill', 1.0], ['churrasco', 1.5], ['ribs', 1.0]]
6
[1.         0.25       0.

[['service', 0.3333333333333333], ['always', 0.5], ['great', 1.0], ['owner', 1.5], ['walks', 1.0], ['around', 0.5], ['make', 0.3333333333333333], ['sure', 0.25], ['enjoy', 0.2]]
[['service', 0.3333333333333333], ['always', 0.5], ['great', 1.0], ['owner', 1.5], ['walks', 1.0], ['around', 0.5], ['make', 0.3333333333333333], ['sure', 0.25], ['enjoy', 0.2]]
9
[0.33333333 0.5        0.5        0.2        0.25       1.
 1.5        0.33333333]
gratuity 272
automatically 272
[['15', 0.25], ['gratuity', 0.3333333333333333], ['automatically', 0.5], ['added', 1.0], ['bill', 1.5]]
[['15', 0.25], ['gratuity', 0.3333333333333333], ['automatically', 0.5], ['added', 1.0], ['bill', 1.5]]
5
[1.5  0.25 1.  ]
veggies 273
sorts 273
eastern 273
falafel 273
medley 273
baclava 273
soooo 273
fooood 273
[['fresh', 0.058823529411764705], ['veggies', 0.0625], ['sorts', 0.06666666666666667], ['middle', 0.07142857142857142], ['eastern', 0.07692307692307693], ['spreads', 0.08333333333333333], ['cheese', 1.0], ['fala

jury 352
duty 352
rode 352
bike 352
centre 352
tacos 352
hibiscus 352
[['jury', 0.0625], ['duty', 0.06666666666666667], ['rode', 0.07142857142857142], ['bike', 0.07692307692307693], ['centre', 0.08333333333333333], ['street', 0.09090909090909091], ['lunch', 0.1], ['break', 0.1111111111111111], ['came', 0.125], ['across', 0.14285714285714285], ['great', 0.16666666666666666], ['little', 0.2], ['place', 0.25], ['awesome', 0.3333333333333333], ['chicken', 0.5], ['tacos', 1.0], ['hibiscus', 1.5], ['lemonade', 1.5]]
[['jury', 0.0625], ['duty', 0.06666666666666667], ['rode', 0.07142857142857142], ['bike', 0.07692307692307693], ['centre', 0.08333333333333333], ['street', 0.09090909090909091], ['lunch', 0.1], ['break', 0.1111111111111111], ['came', 0.125], ['across', 0.14285714285714285], ['great', 0.16666666666666666], ['little', 0.2], ['place', 0.25], ['awesome', 0.3333333333333333], ['chicken', 0.5], ['tacos', 1.0], ['hibiscus', 1.5], ['lemonade', 1.5]]
18
[0.14285714 0.09090909 0.25       0

...

blazing 760
chillis 760
human 760
[['dish', 0.3333333333333333], ['arrived', 0.5], ['blazing', 1.0], ['green', 1.5], ['chillis', 1.5], ['definitely', 1.0], ['not', 0.5], ['edible', 0.3333333333333333], ['human', 0.25]]
[['dish', 0.3333333333333333], ['arrived', 0.5], ['blazing', 1.0], ['green', 1.5], ['chillis', 1.5], ['definitely', 1.0], ['not', 0.5], ['edible', 0.3333333333333333], ['human', 0.25]]
9
[1.         1.5        0.5        0.33333333 0.5        0.33333333]
manners 762
[['several', 0.3333333333333333], ['times', 0.5], ['put', 1.0], ['waiters', 1.5], ['bad', 1.0], ['manners', 0.5], ['knowing', 0.3333333333333333], ['job', 0.25], ['not', 0.2], ['easy', 0.16666666666666666]]
[['several', 0.3333333333333333], ['times', 0.5], ['put', 1.0], ['waiters', 1.5], ['bad', 1.0], ['manners', 0.5], ['knowing', 0.3333333333333333], ['job', 0.25], ['not', 0.2], ['easy', 0.16666666666666666]]
10
[0.33333333 1.5        0.33333333 1.         0.5        0.16666667
 0.2        0.25       1.     

vip 856
[['times', 0.058823529411764705], ['gone', 0.0625], ['romantic', 0.06666666666666667], ['date', 0.07142857142857142], ['heaven', 0.07692307692307693], ['walk', 0.08333333333333333], ['get', 0.09090909090909091], ['booth', 0.1], ['windows', 0.1111111111111111], ['treated', 0.125], ['like', 0.14285714285714285], ['vip', 0.16666666666666666], ['not', 0.2], ['crowded', 0.25], ['place', 0.3333333333333333], ['great', 0.5], ['food', 1.0], ['service', 1.5]]
[['times', 0.058823529411764705], ['gone', 0.0625], ['romantic', 0.06666666666666667], ['date', 0.07142857142857142], ['heaven', 0.07692307692307693], ['walk', 0.08333333333333333], ['get', 0.09090909090909091], ['booth', 0.1], ['windows', 0.1111111111111111], ['treated', 0.125], ['like', 0.14285714285714285], ['vip', 0.16666666666666666], ['not', 0.2], ['crowded', 0.25], ['place', 0.3333333333333333], ['great', 0.5], ['food', 1.0], ['service', 1.5]]
18
[1.5        0.125      0.09090909 0.06666667 0.0625     1.
 0.33333333 0.111111

delivers 964
combine 964
ensure 964
repast 964
[['speedy', 1.0], ['delivers', 1.5], ['great', 1.0], ['food', 0.5], ['decent', 0.3333333333333333], ['prices', 0.25], ['friendly', 0.2], ['service', 0.16666666666666666], ['combine', 0.14285714285714285], ['ensure', 0.125], ['enjoyable', 0.1111111111111111], ['repast', 0.1]]
[['speedy', 1.0], ['delivers', 1.5], ['great', 1.0], ['food', 0.5], ['decent', 0.3333333333333333], ['prices', 0.25], ['friendly', 0.2], ['service', 0.16666666666666666], ['combine', 0.14285714285714285], ['ensure', 0.125], ['enjoyable', 0.1111111111111111], ['repast', 0.1]]
12
[0.16666667 0.25       0.33333333 1.         0.5        0.2
 1.         0.11111111]
flan 965
sopaipillas 965
[['desserts', 1.5], ['include', 1.0], ['flan', 0.5], ['sopaipillas', 0.3333333333333333]]
[['desserts', 1.5], ['include', 1.0], ['flan', 0.5], ['sopaipillas', 0.3333333333333333]]
4
[1.  1.5]
drunk 966
[['good', 0.08333333333333333], ['go', 0.09090909090909091], ['drinks', 0.1], ['not', 0

jokes 1006
friendliness 1006
[['food', 1.5], ['take', 1.0], ['extra', 0.5], ['minutes', 0.3333333333333333], ['come', 0.25], ['but', 0.2], ['cute', 0.16666666666666666], ['waiters', 0.14285714285714285], ['jokes', 0.125], ['friendliness', 0.1111111111111111], ['made', 0.1]]
[['food', 1.5], ['take', 1.0], ['extra', 0.5], ['minutes', 0.3333333333333333], ['come', 0.25], ['but', 0.2], ['cute', 0.16666666666666666], ['waiters', 0.14285714285714285], ['jokes', 0.125], ['friendliness', 0.1111111111111111], ['made', 0.1]]
11
[0.16666667 0.14285714 0.5        1.5        0.1        0.33333333
 0.2        0.25       1.        ]
al 1008
dente 1008
drowned 1008
[['perfectly', 0.125], ['al', 0.14285714285714285], ['dente', 0.16666666666666666], ['pasta', 0.2], ['not', 0.25], ['drowned', 0.3333333333333333], ['sauce', 0.5], ['generous', 1.0], ['portions', 1.5]]
[['perfectly', 0.125], ['al', 0.14285714285714285], ['dente', 0.16666666666666666], ['pasta', 0.2], ['not', 0.25], ['drowned', 0.33333333333

[0.5        1.         0.14285714 0.33333333 0.33333333 0.5
 0.125      1.5        1.        ]
driving 1091
[['starving', 0.5], ['small', 1.0], ['portions', 1.5], ['driving', 1.0], ['crazy', 0.5]]
[['starving', 0.5], ['small', 1.0], ['portions', 1.5], ['driving', 1.0], ['crazy', 0.5]]
5
[1.  0.5 1.5 0.5]
focacchia 1092
[['served', 1.0], ['focacchia', 1.5], ['bread', 1.5], ['die', 1.0]]
[['served', 1.0], ['focacchia', 1.5], ['bread', 1.5], ['die', 1.0]]
4
[1.  1.5 1. ]
unfortunate 1095
quesedilla 1095
[['unfortunate', 0.1111111111111111], ['lady', 0.125], ['next', 0.14285714285714285], ['us', 0.16666666666666666], ['thought', 0.2], ['ordered', 0.25], ['salad', 1.5], ['including', 0.5], ['asking', 1.0], ['dressing', 1.5], ['instead', 1.0], ['given', 0.5], ['quesedilla', 0.3333333333333333]]
[['unfortunate', 0.1111111111111111], ['lady', 0.125], ['next', 0.14285714285714285], ['us', 0.16666666666666666], ['thought', 0.2], ['ordered', 0.25], ['salad', 1.5], ['including', 0.5], ['asking', 1

In [16]:
pf.close()
test_weight_x = np.array(test_weight_x)

print(test_weight_x.shape)

test_f.close()

(1120, 3613)


In [17]:
print(len(test_sentence_weights))

1120
